In [1]:
%pwd

'c:\\Users\\Khanh\\Desktop\\Medic_Chat\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Khanh\\Desktop\\Medic_Chat'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [9]:
extracted_data = load_pdf_file(data='Data/')

Overwriting cache for 0 1156


In [10]:
extracted_data

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2021-05-21T21:42:29-07:00', 'moddate': '2021-05-21T21:42:29-07:00', 'source': 'Data\\benh-hoc.pdf', 'total_pages': 416, 'page': 0, 'page_label': '1'}, page_content='1 \n \nGIỚI THIỆU HỌC PHẦN \nBỆNH HỌC CƠ SỞ \nĐối tượng : Cao đẳng XN, HA, PHCN, Dược \n \nSố tín chỉ :      04 (4/0) \nSố tiết:     60 tiết \n+ Lý thuyết:     60 tiết( 4 tiết/ 1 tuần) \n+ Tự học:    120 giờ \n+ Thực hành :    0 tiết \nThời điểm thực hiện:   Học kỳ IV \n \nMỤC TIÊU HỌC PHẦN : \n1. Trình bày được nguyên nhân, triệu chứng, chẩn đoán và biến chứng một số \nbệnh nội khoa, truyền nhiễm, chuyên khoa  ngoại khoa, nhi khoa, sản khoa thường \ngặp.  \n           2. Trình bày được hướng điều trị và tư vấn phòng bệnh một số bệnh nội khoa, \ntruyền nhiễm, chuyên khoa ngoại khoa, nhi khoa, sản khoa thường gặp.  \n          3. Phân biệt được sự giống nhau, khác nhau về triệu chứng lâm sàng, cận lâm \nsàng của 1 số  bệnh nội khoa, truyền nhiễm, c

In [66]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [67]:
text_chunks=text_split(extracted_data)

print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 1723


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [84]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='dangvantuan/vietnamese-document-embedding', model_kwargs={"trust_remote_code": True})
    return embeddings


In [85]:
embeddings = download_hugging_face_embeddings()

In [75]:
embeddings

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: VietnameseModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [21]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [22]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [23]:
PINECONE_API_KEY 

'pcsk_22zDE5_HqPJ8XzBSBccVcAFbyrpKr3UdKexMwsHahq2d1Y6EfiFVZhGzc7ePMTUyQdGkwM'

In [52]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_22zDE5_HqPJ8XzBSBccVcAFbyrpKr3UdKexMwsHahq2d1Y6EfiFVZhGzc7ePMTUyQdGkwM")

index_name = "medicalbotvienamese"


pc.create_index(
    name=index_name,
    dimension=768, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [25]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_22zDE5_HqPJ8XzBSBccVcAFbyrpKr3UdKexMwsHahq2d1Y6EfiFVZhGzc7ePMTUyQdGkwM"
os.environ["GROQ_API_KEY"] = "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"


In [86]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [87]:
docsearch

In [88]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [89]:
retrieved_docs = retriever.invoke("VIÊM PHẾ QUẢN CẤP là gì?")

In [90]:
retrieved_docs

[Document(id='a18545f8-c00d-49e1-8a28-11a6e4e4ecf8', metadata={'creationdate': '2021-05-21T21:42:29-07:00', 'creator': 'PyPDF', 'moddate': '2021-05-21T21:42:29-07:00', 'page': 385.0, 'page_label': '386', 'producer': 'PyPDF', 'source': 'Data\\benh-hoc.pdf', 'total_pages': 416.0}, page_content="386 \n \nBài 103 \nVIÊM PHẾ QUẢN PHỔI  \n \n \nMỤC TIÊU: \n1. Trình bày được nguyên nhân, triệu chứng, chẩn đoán bệnh viêm phế quản \nphổi.(VPQP ) \n2. Trình bày được tiến triển và biến chứng của bệnh VPQP . \n3. Trình bày được phác đồ điều trị VPQP \n \nNỘI DUNG: \n 1.  Đại cương \n- Danh từ 'viêm phế quản phổi' để chỉ tổn thương viêm các phế quản và túi phổi \n(phế nang) rải rác cả hai bên phổi làm rối loạn trao đổi khí dễ gây SHH. \n- Bệnh hay gặp ở trẻ nhỏ (<1 tuổi). \n2.  Nguyên nhân:"),
 Document(id='d7c74862-538d-4daa-bd0f-e0b1eb25fddc', metadata={'creationdate': '2021-05-21T21:42:29-07:00', 'creator': 'PyPDF', 'moddate': '2021-05-21T21:42:29-07:00', 'page': 400.0, 'page_label': '401', 'pro

In [ ]:
from groq import Groq
from langchain.chat_models import ChatOpenAI
import os
llm = ChatOpenAI(
    model="llama-3.3-70b-versatile",
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key="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",
)


In [103]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. "
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [104]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [105]:
response = rag_chain.invoke({"input": "Viêm phế  quản  cấp là gì"})
print(response["answer"])

Viêm phế quản cấp là tình trạng viêm nhiễm niêm mạc của phế quản lớn và phế quản trung bình. Bệnh có thể xảy ra ở bất kỳ tuổi nào, nhưng gặp nhiều ở trẻ em và người già, thường vào mùa lạnh và đầu mùa xuân. Biểu hiện lâm sàng chính là ho.


In [106]:
response = rag_chain.invoke({"input": "điều trị Viêm phế quản"})
print(response["answer"])

Điều trị viêm phế quản cấp bao gồm các bước sau:

1. **Thể nhẹ**: 
 - Nghỉ ngơi tại giường
 - Uống đủ nước
 - Cho Codein (để giảm ho)
 - Không cần dùng kháng sinh

2. **Thể nặng**: 
 - Có các triệu chứng như viêm mũi mủ, viêm amiđan, viêm VA, viêm tai giữa, viêm phế quản
 - Cho kháng sinh:
   - Nhóm Macrolid: 
     - Erythromycin uống 2g/ngày
     - Azithromycin 0.5 g/ngày đầu sau đó 250mg x 4 ngày
   - Nhóm Quinolon: 
     - Ciprofloxacin uống 200 - 400mg/ ngày
     - Biseptol 0,48g x 4 viên/ngày cho 10 ngày
 - Nên cho kháng histamin khi có dấu hiệu co thắt phế quản
 - Hạ sốt, giảm đau
 - Long đờm: Acemux, Mucomys 200 mg x 4 gói/ ngày
 - Điều trị nguyên nhân

3. **Phòng bệnh**:
 - Loại bỏ các yếu tố kích thích: tránh khói bụi, môi trường ô nhiễm
 - Giữ gìn sức khoẻ, giữ ấm trong mùa lạnh
 - Tiêm vaccin chống virus, vi khuẩn
 - Điều trị tốt ổ nhiễm khuẩn ở vùng tai mũi họng
 - Loại bỏ yếu tố kích thích có hại: bỏ thuốc lá, thuốc lào
 - Giữ ấm cổ, ngực trong mùa lạnh

Lưu ý: Điều trị cầ

In [109]:
response = rag_chain.invoke({"input": "Nguyên nhân gây nên Viên phế quản cấp"})
print(response["answer"])

Nguyên nhân gây nên Viêm phế quản cấp chủ yếu là do nhiễm khuẩn và vi rút. Cụ thể, các nguyên nhân bao gồm:

- Nhiễm khuẩn đường hô hấp trên do vi khuẩn và vi rút: viêm mũi, viêm xoang, viêm amidan.
- Do bệnh truyền nhiễm: cúm, sởi, ho gà.
- Hít phải hơi độc: Chlore, amoniac, khói thuốc lá, dung môi công nghiệp.
- Yếu tố dị ứng: Cơ địa dị ứng.

Tuy nhiên, cần lưu ý rằng thông tin này không chỉ liên quan đến viêm phế quản cấp, mà còn liên quan đến các bệnh khác như hen phế quản và tâm phế mạn. Để trả lời chính xác, cần có thông tin cụ thể hơn về viêm phế quản cấp.


In [39]:
import os
from dotenv import load_dotenv, find_dotenv


load_dotenv('/Users/Khanh/Desktop/Medic_Chat/.env')
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
print(f"PINECONE_API_KEY: {PINECONE_API_KEY}")

PINECONE_API_KEY: pcsk_22zDE5_HqPJ8XzBSBccVcAFbyrpKr3UdKexMwsHahq2d1Y6EfiFVZhGzc7ePMTUyQdGkwM


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv


load_dotenv('/Users/Khanh/Desktop/Medic_Chat/.env')
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
print(f"PINECONE_API_KEY: {PINECONE_API_KEY}")